In [1]:
import requests

In [2]:
import re

In [3]:
import csv

In [4]:
from bs4 import BeautifulSoup

In [15]:
urllist = list()

In [29]:
incre = 1
with open("rank.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',')
    for i in range(1,22):
        params = {"page":i}
        r = requests.get("http://www.zinch.cn/ss/us",params = params)
        soup = BeautifulSoup(r.text,"html.parser")
        for tag in soup.find_all("h2"):
            if "class" in list(tag.attrs.keys()):
                if "properties-name-en" in tag['class']:
                    urllist.append(tag.findNext().attrs['href']);
                    if tag.findNext().text.find('(') == -1:
                        spamwriter.writerow([incre,tag.findNext().text])
                    else:
                        ind = tag.findNext().text.find('(')
                        spamwriter.writerow([incre,tag.findNext().text[0:ind-1]])
                    incre = incre+1
                        

In [64]:
logolist = list()

In [40]:
from urllib.request import urlretrieve

In [65]:
for i in range(1,22):
    params = {"page":i}
    r = requests.get("http://www.zinch.cn/ss/us",params = params)
    soup = BeautifulSoup(r.text,"html.parser")
    for tag in soup.find_all("img"):
        if "class" in list(tag.attrs.keys()):
            if "imagecache-75x75" in tag['class']:
                logolist.append(tag.attrs['src']);

In [81]:
## Crawling the school logo image
for i in range(0,len(urllist)):
    fileurl = "./schoollogo/"+urllist[i][1:]+".jpg"
    urlretrieve(logolist[i],fileurl);